In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import networkx as nx
import matplotlib.pyplot as plt
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

import pandas as pd
import glob
import os
import requests
import json
import pprint
from os.path import exists
import sys

C:\Users\bja\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\bja\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
C:\ProgramData\KB\Temp\ipykernel_30812\2203745756.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nlt

In [82]:
def exit_program(m="Exiting the program..."):
    print(m)
    sys.exit(0)

def get_keywords(text):
    url = "https://cst.dk/tools/index.php#output"

    headers= {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Content-Type": "multipart/form-data; boundary=----WebKitFormBoundaryrlSsbJBaMLa5XHqW",

    }
    #"Cookie": "PHPSESSID=tu6ml4qumb5cnktdcdn0ptmqkb; _pk_id.3.76c1=5103dc170a49ee5e.1708185635.; _pk_ses.3.76c1=1",
    data='------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\nContent-Disposition: form-data; name="MAX_FILE_SIZE"\r\n\r\n8000000'
    data=data+'\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\nContent-Disposition: form-data; name="inputform"\r\n\r\nj'
    data=data+'\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\nContent-Disposition: form-data; name="language"\r\n\r\nda'
    data=data+'\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\nContent-Disposition: form-data; name="password"\r\n\r\n'
    data=data+'\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\nContent-Disposition: form-data; name="inputText"\r\n\r\n'
    data=data+text
    data=data+'\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\nContent-Disposition: form-data; name="inputFile"; '
    data=data+'filename=""\r\nContent-Type: application/octet-stream\r\n\r\n\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\n'
    data=data+'Content-Disposition: form-data; name="token"\r\n\r\nj\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\n'
    data=data+'Content-Disposition: form-data; name="pos"\r\n\r\nj\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\n'
    data=data+'Content-Disposition: form-data; name="lemma"\r\n\r\nj\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\n'
    data=data+'Content-Disposition: form-data; name="keyword"\r\n\r\nj\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\n'
    data=data+'Content-Disposition: form-data; name="anonym"\r\n\r\nn\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\n'
    data=data+'Content-Disposition: form-data; name="abbr"\r\n\r\nj\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\n'
    data=data+'Content-Disposition: form-data; name="mwu"\r\n\r\nj\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\n'
    data=data+'Content-Disposition: form-data; name="what"\r\n\r\nbw\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\n'
    data=data+'Content-Disposition: form-data; name="sorting"\r\n\r\nfwt\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW\r\n'
    data=data+'Content-Disposition: form-data; name="dict"\r\n\r\nj\r\n------WebKitFormBoundaryrlSsbJBaMLa5XHqW--\r\n'
    data=data.encode()
    response = requests.post(url, data=data, headers=headers)

    from bs4 import BeautifulSoup
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    result=[]
    table = soup.find("table", { "class" : "result" })
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        weight = cells[0].findAll(text=True)
        keyword = str(cells[1].findAll(text=True)).replace(',','-')
        if not str(weight) == "['Vægt']":
            result.append([str(weight)[2:][:-2],str(keyword)[4:][:-2]])
    return(result)

def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    stopwords = nltk.corpus.stopwords.words('danish')
    newStopWords = ['gerne', 'sige', 'så', 'kan', 'vores', 'ved', 'lave', 'se', 'ting', 'ja']
    for i in newStopWords:
        stopwords.append(i)
    newStopWords = ['nej', 'øh', 'måske', 'syg', 'en', 'to', 'lidt', 'nok', 'samt', 'én']
    for i in newStopWords:
        stopwords.append(i)
    stop_words=set(stopwords)
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    return ' '.join(tokens)

def extract_top_terms(text, vectorizer, n=5):
    tfidf_matrix = vectorizer.fit_transform([text])
    feature_array = vectorizer.get_feature_names_out()
    tfidf_sorting = tfidf_matrix.toarray().flatten().argsort()[::-1]
    top_n_terms = feature_array[tfidf_sorting][:n]
    return ', '.join(top_n_terms)

def add_edges(graph, terms):
    for i in range(len(terms)):
        for j in range(i+1, len(terms)):
            if graph.has_edge(terms[i], terms[j]):
                graph[terms[i]][terms[j]]['weight'] += 1
            else:
                graph.add_edge(terms[i], terms[j], weight=1)

def merge_nodes(G,a,b,h1='STRATEGI2024',h2='MORGENMØDE2024',h3='STRATEGIMORGENMØDE',set_extra_attr3=False):
    GO=G
    if a in G.nodes and b in G.nodes:
        print("merger",a,"og",b)
        H=nx.contracted_nodes(G,a,b, self_loops=True, copy=True)
        if set_extra_attr3:
            
            if GO.nodes[a]['attr3'] == h1 and GO.nodes[b]['attr3'] == h2:
                H.nodes[a]['attr3'] = h3
            if GO.nodes[b]['attr3'] == h1 and GO.nodes[a]['attr3'] == h2:
                H.nodes[a]['attr3'] = h3
            if GO.nodes[a]['attr3'] == h3 or GO.nodes[b]['attr3'] == h3:
                H.nodes[a]['attr3'] = h3
            print("sætter attr3 til ",H.nodes[a]['attr3'])
        return H
    else:
        return G
    
def merge_nodes2(G,a,b,h1='STRATEGI2024',h2='MORGENMØDE2024',h3='STRATEGIMORGENMØDE',set_extra_attr3=False):
    if b in G.nodes:
        #print("merger",a,"og",b)
        mapping = {b:a}
        G = nx.relabel_nodes(G, mapping)
    return G

def compare_graphs(file1,file2,h1='STRATEGI2024',h2='MORGENMØDE2024',h3='STRATEGIMORGENMØDE',titel1='',titel2='',outfile='network.gexf',kw=False):                
    write_temp(file1,"temp1.txt")
    write_temp(file2,"temp2.txt")
    if kw is True:
        fn1="dataframes/"+file1+".kw.df.csv"
        fn2="dataframes/"+file2+".kw.df.csv"
    else:
        fn1="dataframes/"+file1+".df.csv"
        fn2="dataframes/"+file2+".df.csv"
    
    fn1=fn1.replace('*','_')
    fn2=fn2.replace('*','_')
    print("sammenligner: ", fn1, " med ",fn2)

    n=8 #number of terms to extract per line    

    if not exists(fn1):
        df = pd.read_csv("temp1.txt", encoding='UTF-8', sep=';', names=['Text'])
        df['Processed_Text'] = df['Text'].apply(preprocess_text)

        # OPTION FOR ADJUSTING NGRAM RANGE - 1,1 = Unigrams - 1,2 - Unigrams and bigrams - 1,3 - Unigrams, bigrams and trigrams
        # Can also be replaces with countvectorizer instead, which just counts all occurences across the corpus instead of ranking words occuring frequently in a smaller set of documents more highly
        vectorizer = TfidfVectorizer(ngram_range=(1, 1)) # CountVectorizer

        # OPTION TO ADJUST HOW MANY TERMS ARE BEING EXTRACTED - DEFAULT 5
        df['Top_Terms'] = df['Processed_Text'].apply(lambda x: extract_top_terms(x, vectorizer, n=n))
        if kw is True:
            leng=len(df['Processed_Text'])-1
            for n in range(0,len(df['Processed_Text'])-1,1):
                kw=get_keywords(df['Processed_Text'][n])
                li=''
                for w,k in kw:
                    li=li+k+', '
                df.at[n, "Top_Terms"] = li
        df.to_csv(fn1)
    else:
        df = pd.read_csv(fn1)
    
    if not exists(fn2):
        df2 = pd.read_csv("temp2.txt", encoding='UTF-8', sep=';', names=['Text'])
        df2['Processed_Text'] = df2['Text'].apply(preprocess_text)

        # OPTION FOR ADJUSTING NGRAM RANGE - 1,1 = Unigrams - 1,2 - Unigrams and bigrams - 1,3 - Unigrams, bigrams and trigrams
        # Can also be replaces with countvectorizer instead, which just counts all occurences across the corpus instead of ranking words occuring frequently in a smaller set of documents more highly
        vectorizer = TfidfVectorizer(ngram_range=(1, 1)) # CountVectorizer

        # OPTION TO ADJUST HOW MANY TERMS ARE BEING EXTRACTED - DEFAULT 5
        df2['Top_Terms'] = df2['Processed_Text'].apply(lambda x: extract_top_terms(x, vectorizer, n=n))
        if kw is True:
            leng=len(df2['Processed_Text'])-1
            for n in range(0,len(df2['Processed_Text'])-1,1):
                kw=get_keywords(df2['Processed_Text'][n])
                li=''
                for w,k in kw:
                    li=li+k+', '
                df2.at[n, "Top_Terms"] = li
        df2.to_csv(fn2)
    else:
        df2 = pd.read_csv(fn2)
    
    G = nx.Graph()
    G1 = nx.Graph()
    G2 = nx.Graph()

    for index, row in df.iterrows():
        if isinstance(row['Top_Terms'], str):
            top_terms = row['Top_Terms'].split(', ')
            add_edges(G1, top_terms)
            for t in top_terms:
                if not G1.has_edge(t, h1):
                    G1.add_edge(t, h1, weight=1)

    for index, row in df2.iterrows():
        if isinstance(row['Top_Terms'], str):
            top_terms = row['Top_Terms'].split(', ')
            add_edges(G2, top_terms)
            for t in top_terms:
                if not G2.has_edge(t, h2):
                    G2.add_edge(t, h2, weight=1)
                    #print("laver edge:",t,h2)
    
    file1 = open('stemming.txt', 'r', encoding='UTF-8')
    Lines = file1.readlines()
    for line in Lines:
        words=line.strip().split(',')
        for q in range(0,len(words)-1,1):
            for w in range(q,len(words),1):
                if not q == w:
                    #print("#### Graf 1")
                    G1=merge_nodes2(G1,words[q].strip(),words[w].strip(),h1,h2,h3,False)
                    #print("#### Graf 2")
                    G2=merge_nodes2(G2,words[q].strip(),words[w].strip(),h1,h2,h3,False)
                    
                    
    G = nx.compose(G1,G2)

    for n in G.nodes:
        if G.has_edge(n,h1): #har kan til nummer 1
            if G.has_edge(n, h2): #har også kant til nummer 2 - dvs. findes i begge
                G.nodes[n]["attr3"]= h3 #får tag for fælles
                #print("fælles node:",n)
            else: #har kun kant til nummer 1
                G.nodes[n]["attr3"]= h1
        else:
            if G.has_edge(n, h2):  #findes kun i nummer 2
                G.nodes[n]["attr3"]= h2
            #else:
                #print("der er noget galt med node:",n)
    
    file1 = open('stemming.txt', 'r', encoding='UTF-8')
    Lines = file1.readlines()

    for line in Lines:
        words=line.strip().split(',')
        for q in range(0,len(words)-1,1):
            for w in range(q,len(words),1):
                if not q == w:
                    #print("#### Fælles graf:",words[q].strip(),words[w].strip())
                    G=merge_nodes(G,words[q].strip(),words[w].strip(),h1,h2,h3,True)
    
    print(len(G1.nodes)-1)
    print(len(G2.nodes)-1)
    
    t=0
    t2=0
    o=0
    o2=0
    s=0
    s2=0
    for n in G.nodes:
        t=t+G.degree(n)
        t2=t2+1
        #print("længde af ord: ",len(n),n)
        if not len(n) == 0:
            if 'attr3' in G.nodes[n]: #noden har sat attr3 (gælder for alle noder minus psudonoderne = de 2 store)
                if G.nodes[n]['attr3'] == h3: #findes i begge
                    #print("ORD:",n,"#")
                    #print(G.nodes[n]['attr3'])
                    if n in G1.nodes:
                        tv=G1.degree(n)
                    else:
                        tv=G.degree(n)
                        #o=o+1
                        print("bruger tallet:",G.degree(n),"til node",n)
                    o=o+tv
                    o2=o2+1
                    print(n," - score:",tv," - samlet score: ",o)
                if h1 in G.nodes[n]['attr3'] or h3 in G.nodes[n]['attr3']:
                    s=s+G1.degree(n)
                    s2=s2+1
    if kw is True:
        es='(on CST Keywords)'
    else:
        es='(on TF-IDF Top_Terms)'
    print(o,s,o/s*100,"%", "overlap mellem",titel1,"&",titel2," målt på nodernes degree i ",titel1,es)
    print(o2,s2,o2/s2*100,"%", "overlap mellem",titel1,"&",titel2," målt på antal noder i ",titel1,es)
    print(o,t,o/t*100,"%", "overlap totalt",titel1,"&",titel2," målt alle noders degree",es)
    print(o2,t2,o2/t2*100,"%", "overlap totalt",titel1,"&",titel2," målt på antal noder totalt",es)
    
    # OPTION TO FILTER ON NETWORK BEFORE GEPHI - Primarily a concern if the network is too big
    node_frequency_threshold = 1  # Minimum number of occurrences for a node
    edge_frequency_threshold = 1  # Minimum weight for an edge

    nodes_with_low_frequency = [node for node, degree in dict(G.degree(weight='weight')).items() if degree < node_frequency_threshold]
    G.remove_nodes_from(nodes_with_low_frequency)

    edges_with_low_frequency = [(u, v) for u, v, data in G.edges(data=True) if data['weight'] < edge_frequency_threshold]
    G.remove_edges_from(edges_with_low_frequency)

    #Removing isolated nodes - can be disabled by commenting these lines
    isolated_nodes = list(nx.isolates(G))
    G.remove_nodes_from(isolated_nodes)

    # OPTION TO PLOT THE NETWORK within Jupyter Notebook - comment out if you only want to work in Gephi with GEXF-format
    #plt.figure(figsize=(10, 10))
    #pos = nx.spring_layout(G, k=0.5)  # k regulates the distance between nodes
    #nx.draw_networkx(G, pos, with_labels=True, node_color='skyblue', font_size=10, edge_color='gray')
    #plt.title('Refined Keyword Co-Occurrence Network')
    #plt.axis('off')  # Turn off the axis
    #plt.show()
    if kw is True: #CST Keywords
        outfile = outfile.replace('.gexf','-kw.gexf')
    outfile="output/"+outfile
    nx.write_gexf(G, outfile)
    print("GEXF-fil gemt til: ", outfile)
    print('#####################')

def write_temp(regexp='',outfile='temp.txt'): #join multiple text-files into one
    path = r'C:\Users\bja\OneDrive - Det Kgl. Bibliotek\MDO\Masterprojekt 4. semester' # use your path
    all_files = glob.glob(os.path.join(path , regexp))
    if len(all_files) == 0: ## only singlefile (or file(s) not found at all)
        fn=path+"\\"+regexp
        if not exists(fn):
            exit_program('file not found: '+fn)
        all_files=[fn]
    print(all_files)
    with open(outfile, 'w') as outfile:
        for fname in all_files:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)

keyw=False #change to True to generate graphs based on CST-keywords

compare_graphs('*efter*.txt','*råd*.txt','INTERVIEWS','SPØRGESKEMA','INTERVIES-SPG',"Interviews","Spørgeskema", 'interviews-spørgeskema-efter.gexf', keyw)

compare_graphs('strategi2.txt','morgenmøde2.txt','STRATEGI2024','MORGENMØDE2024','STRATEGIMORGENMØDE',"Strategi","Morgenmøde", 'strategi-morgenmøde.gexf',keyw)

compare_graphs('strategi2.txt','leder*1*strategi*.txt','STRATEGI2024','LEDER1-2024','STRATEGILEDER1',"Strategi","Leder-1", 'strategi-leder1.gexf', keyw)
compare_graphs('strategi2.txt','leder*2*strategi*.txt','STRATEGI2024','LEDER2-2024','STRATEGILEDER2',"Strategi","Leder-2", 'strategi-leder2.gexf', keyw)
compare_graphs('strategi2.txt','leder*3*strategi*.txt','STRATEGI2024','LEDER3-2024','STRATEGILEDER3',"Strategi","Leder-3", 'strategi-leder3.gexf', keyw)
compare_graphs('strategi2.txt','leder*4*strategi*.txt','STRATEGI2024','LEDER4-2024','STRATEGILEDER4',"Strategi","Leder-4", 'strategi-leder4.gexf', keyw)
compare_graphs('strategi2.txt','leder*5*strategi*.txt','STRATEGI2024','LEDER5-2024','STRATEGILEDER5',"Strategi","Leder-5", 'strategi-leder5.gexf', keyw)
compare_graphs('strategi2.txt','leder*6*strategi*.txt','STRATEGI2024','LEDER6-2024','STRATEGILEDER6',"Strategi","Leder-6", 'strategi-leder6.gexf', keyw)


compare_graphs('strategi2.txt','medarbejder*1*strategi*.txt','STRATEGI2024','MEDARBEJDER1-2024','STRATEGIMEDARBEJDER1',"Strategi","Medarbejder-1", 'strategi-medarbejder1.gexf', keyw)
compare_graphs('strategi2.txt','medarbejder*2*strategi*.txt','STRATEGI2024','MEDARBEJDER2-2024','STRATEGIMEDARBEJDER2',"Strategi","Medarbejder-2", 'strategi-medarbejder2.gexf', keyw)
compare_graphs('strategi2.txt','medarbejder*3*strategi*.txt','STRATEGI2024','MEDARBEJDER3-2024','STRATEGIMEDARBEJDER3',"Strategi","Medarbejder-3", 'strategi-medarbejder3.gexf', keyw)
compare_graphs('strategi2.txt','medarbejder*4*strategi*.txt','STRATEGI2024','MEDARBEJDER4-2024','STRATEGIMEDARBEJDER4',"Strategi","Medarbejder-4", 'strategi-medarbejder4.gexf', keyw)
compare_graphs('strategi2.txt','medarbejder*5*strategi*.txt','STRATEGI2024','MEDARBEJDER5-2024','STRATEGIMEDARBEJDER5',"Strategi","Medarbejder-5", 'strategi-medarbejder5.gexf', keyw)
compare_graphs('strategi2.txt','medarbejder*6*strategi*.txt','STRATEGI2024','MEDARBEJDER6-2024','STRATEGIMEDARBEJDER6',"Strategi","Medarbejder-6", 'strategi-medarbejder6.gexf', keyw)

compare_graphs('strategi2.txt','medarbejder*strategi*.txt','STRATEGI2024','MEDARBEJDERE2024','STRATEGIMEDARBEJDERE',"Strategi","Alle medarbejdere",'strategi-alle_medarbejdere.gexf', keyw)

compare_graphs('strategi2.txt','leder*strategi*.txt','STRATEGI2024','LEDERE2024','STRATEGILEDERE',"Strategi","Alle ledere",'strategi-alle_ledere.gexf', keyw)

compare_graphs('strategi2.txt','*er*strategi*.txt','STRATEGI2024','ALLE2024','STRATEGIALLE',"Strategi","Alle medarbejdere og ledere",'strategi-alle-deltagere.gexf', keyw)

['C:\\Users\\bja\\OneDrive - Det Kgl. Bibliotek\\MDO\\Masterprojekt 4. semester\\strategi2.txt']
['C:\\Users\\bja\\OneDrive - Det Kgl. Bibliotek\\MDO\\Masterprojekt 4. semester\\leder-5_plain - strategi.txt']
sammenligner:  dataframes/strategi2.txt.df.csv  med  dataframes/leder_5_strategi_.txt.df.csv
merger beskyttelse og beskyttet
sætter attr3 til  STRATEGI2024
347
130
vise  - score: 18  - samlet score:  18
viden  - score: 51  - samlet score:  69
strategi  - score: 14  - samlet score:  83
bibliotek  - score: 116  - samlet score:  199
står  - score: 8  - samlet score:  207
verden  - score: 14  - samlet score:  221
tage  - score: 7  - samlet score:  228
videnssamfund  - score: 11  - samlet score:  239
demokratisk  - score: 17  - samlet score:  256
måde  - score: 14  - samlet score:  270
finde  - score: 26  - samlet score:  296
mission  - score: 5  - samlet score:  301
folkebiblioteker  - score: 30  - samlet score:  331
niveau  - score: 8  - samlet score:  339
fysisk  - score: 34  - saml

In [86]:
print("*leder*2*strategi*.txt.df.csv".replace('*','_'))

_leder_2_strategi_.txt.df.csv


# Byg bi-parite graf med alle deltageres samlede ovarlap men med noder for hver af de 12 deltagere individuelt

In [167]:
path = r'C:\Users\bja\OneDrive - Det Kgl. Bibliotek\MDO\Masterprojekt 4. semester\speciale\strategi-husket-alle-deltagere.gexf' # use your path

#all_files = glob.glob(os.path.join(path , 'strategi-husket-alle-deltagere.gexf'))
#print(all_files[0])

G = nx.read_gexf(path)
G.remove_edges_from(list(G.edges)) #fjerner relationerne i det oprindelige netværk


def add_nodes_from_participant(G,path,st1,st2):
    G2 = nx.read_gexf(path)

    for n in list(G.nodes):
        #print(n.strip())
        if G2.has_edge(n.strip(),st2):
            #print(st1,"har kant til:",n)
            if G.has_edge(st1, n.strip()):
                print("hæver vægten")
                G[st1][n.strip()]['weight'] += 1
            else:
                G.add_edge(st1, n.strip(), weight=1)
                print("tilføjer til:",st1,"-",n.strip())
                
add_nodes_from_participant(G,'output/strategi-medarbejder1.gexf','MEDARBEJDER-1','MEDARBEJDER1-2024')
add_nodes_from_participant(G,'output/strategi-medarbejder2.gexf','MEDARBEJDER-2','MEDARBEJDER2-2024')
add_nodes_from_participant(G,'output/strategi-medarbejder3.gexf','MEDARBEJDER-3','MEDARBEJDER3-2024')
add_nodes_from_participant(G,'output/strategi-medarbejder4.gexf','MEDARBEJDER-4','MEDARBEJDER4-2024')
add_nodes_from_participant(G,'output/strategi-medarbejder5.gexf','MEDARBEJDER-5','MEDARBEJDER5-2024')
add_nodes_from_participant(G,'output/strategi-medarbejder6.gexf','MEDARBEJDER-6','MEDARBEJDER6-2024')

add_nodes_from_participant(G,'output/strategi-leder1.gexf','LEDER-1','LEDER1-2024')
add_nodes_from_participant(G,'output/strategi-leder2.gexf','LEDER-2','LEDER2-2024')
add_nodes_from_participant(G,'output/strategi-leder3.gexf','LEDER-3','LEDER3-2024')
add_nodes_from_participant(G,'output/strategi-leder4.gexf','LEDER-4','LEDER4-2024')
add_nodes_from_participant(G,'output/strategi-leder5.gexf','LEDER-5','LEDER5-2024')
add_nodes_from_participant(G,'output/strategi-leder6.gexf','LEDER-6','LEDER6-2024')



outfile="output/bi-partite-network-uden-originale-relationer.gexf"
nx.write_gexf(G, outfile)

tilføjer til: MEDARBEJDER-1 - viden
tilføjer til: MEDARBEJDER-1 - vej
tilføjer til: MEDARBEJDER-1 - strategi
tilføjer til: MEDARBEJDER-1 - verden
tilføjer til: MEDARBEJDER-1 - rum
tilføjer til: MEDARBEJDER-1 - videre
tilføjer til: MEDARBEJDER-1 - kalder
tilføjer til: MEDARBEJDER-1 - mission
tilføjer til: MEDARBEJDER-1 - vision
tilføjer til: MEDARBEJDER-1 - fyrtårn
tilføjer til: MEDARBEJDER-1 - mål
tilføjer til: MEDARBEJDER-1 - tre
tilføjer til: MEDARBEJDER-1 - skabe
tilføjer til: MEDARBEJDER-1 - national
tilføjer til: MEDARBEJDER-1 - lokal
tilføjer til: MEDARBEJDER-1 - bibliotekspartner
tilføjer til: MEDARBEJDER-1 - bøger
tilføjer til: MEDARBEJDER-1 - år
tilføjer til: MEDARBEJDER-1 - flere
tilføjer til: MEDARBEJDER-1 - forhold
tilføjer til: MEDARBEJDER-1 - læring
tilføjer til: MEDARBEJDER-1 - tredje
tilføjer til: MEDARBEJDER-1 - studerende
tilføjer til: MEDARBEJDER-1 - tværgående
tilføjer til: MEDARBEJDER-1 - organisation
tilføjer til: MEDARBEJDER-1 - pejlemærke
tilføjer til: MEDARBEJD

In [166]:
df2 = pd.read_csv('C:\\Users\\bja\\OneDrive - Det Kgl. Bibliotek\\MDO\\Masterprojekt 4. semester\\speciale\\strategi-morgenmøde-noder-beregninger.csv')
df2
a = df2['modularity_class'].unique()
f = open("clusters-til-lagkager.csv", "w")
for i in sorted(a):
    strategi=0
    morgenmøde=0
    begge=0
    strategi_bc=0
    morgenmøde_bc=0
    begge_bc=0
    
    total=0
    total_bc=0
    
    #ekstra='betweenesscentrality'
    #ekstra='closnesscentrality'
    ekstra='harmonicclosnesscentrality'
    
    df3= df2.loc[df2['modularity_class'] == i]
    df1 = df3[['0',ekstra]]
    for index, row in df1.iterrows():
        c=row['0']
        bc=row[ekstra]
        #print(type(c))
        #print(c)
        if not bc == ekstra:
            if c == 'STRATEGIMORGENMØDE':
                begge=begge+1
                begge_bc=begge_bc+bc
            if c == 'STRATEGI2024':
                strategi=strategi+1
                strategi_bc=strategi_bc+bc

            if c == 'MORGENMØDE2024':
                morgenmøde=morgenmøde+1
                morgenmøde_bc=morgenmøde_bc+bc
            total=total+1
            total_bc=total_bc+bc
    
    print("strategi:",strategi,"-",strategi/total*100)
    print("strategi",ekstra,":",strategi_bc,"-",strategi_bc/total_bc*100)
    print("morgenmøde:",morgenmøde,"-",morgenmøde/total*100)
    print("morgenmøde",ekstra,":",morgenmøde_bc,"-",morgenmøde_bc/total_bc*100)
    print("begge",begge,"-",begge/total*100)
    print("begge",ekstra,":",begge_bc,"-",begge_bc/total_bc*100)
    print("antal noder i cluster",i,"=",total)
    print("total",ekstra,"i cluster",i,"=",total_bc)
    print("##########################")
    f.write(str(i))
    f.write(',')
    f.write(str(total))
    f.write(',')
    f.write(str(strategi))
    f.write(',')
    f.write(str(morgenmøde))
    f.write(',')
    f.write(str(begge))
    f.write(',')
    f.write(str(strategi/total))
    f.write(',')
    f.write(str(morgenmøde/total))
    f.write(',')
    f.write(str(begge/total))
    f.write(',')
    f.write('\n')
    
f.close()
df3

strategi: 13 - 25.49019607843137
strategi harmonicclosnesscentrality : 4.577033 - 23.98929132465009
morgenmøde: 26 - 50.98039215686274
morgenmøde harmonicclosnesscentrality : 9.419474 - 49.369647522962374
begge 12 - 23.52941176470588
begge harmonicclosnesscentrality : 5.0829770000000005 - 26.641061152387568
antal noder i cluster 0 = 51
total harmonicclosnesscentrality i cluster 0 = 19.079483999999994
##########################
strategi: 17 - 26.984126984126984
strategi harmonicclosnesscentrality : 6.281383999999997 - 25.76997187138086
morgenmøde: 30 - 47.61904761904761
morgenmøde harmonicclosnesscentrality : 11.198286000000003 - 45.942027302848906
begge 16 - 25.396825396825395
begge harmonicclosnesscentrality : 6.895149 - 28.288000825770236
antal noder i cluster 1 = 63
total harmonicclosnesscentrality i cluster 1 = 24.374819
##########################
strategi: 9 - 23.076923076923077
strategi harmonicclosnesscentrality : 3.304684 - 21.99636389631527
morgenmøde: 12 - 30.76923076923077
m

,Id,Label,0,1,modularity_class,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality
253,metadata,metadata,STRATEGIMORGENMØDE,NaN,17,4,0.387936,0.414408,1561.867889
332,gå,gå,STRATEGIMORGENMØDE,NaN,17,4,0.358935,0.378864,152.913020
410,mening,mening,MORGENMØDE2024,NaN,17,4,0.375871,0.396814,326.770214
416,kb,kb,MORGENMØDE2024,NaN,17,4,0.415038,0.449358,4396.540418
433,årsplaner,årsplaner,MORGENMØDE2024,NaN,17,4,0.331599,0.345934,0.000000
434,planen,planen,MORGENMØDE2024,NaN,17,4,0.331599,0.345934,0.000000
435,derhen,derhen,MORGENMØDE2024,NaN,17,4,0.344980,0.363766,273.707464
485,syv,syv,MORGENMØDE2024,NaN,17,4,0.326350,0.342606,0.000000
486,systemkonfiguration,systemkonfiguration,MORGENMØDE2024,NaN,17,4,0.326350,0.342606,0.000000
487,seks,seks,MORGENMØDE2024,NaN,17,4,0.326350,0.342606,0.000000


In [ ]:
seen = set()
with open('mails.txt', encoding='UTF-8') as infile:
    with open('mails-unikke.txt', 'w', encoding='UTF-8') as outfile:
        for line in infile:
            if 'kb.dk' not in line.lower():
                print(line)
            if line not in seen:
                outfile.write(line)
                seen.add(line)

from random import random
lines = [line for line in open("mails-unikke.txt", encoding='UTF-8') if random() >= .80]
print(len(lines))
for l in lines:
    print(l)